In [1]:
import pandas as pd
import sqlite3

#Step 1: Load CSV Data
orders = pd.read_csv("orders.csv")
#Step 2: Load JSON Data
users = pd.read_json("users.json")
#Step 2: Load JSON Data
conn = sqlite3.connect(":memory:")
with open("restaurants.sql") as f:
    conn.executescript(f.read())
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
#Step 4: Merge the Data
#Perform joins using keys:
#orders.user_id → users.user_id
#orders.restaurant_id → restaurants.restaurant_id
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")
#Step 5: Create Final Dataset
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)


In [2]:
#Order trends over time

#Orders per month
df["month"] = df["order_date"].dt.to_period("M")
df.groupby("month")["order_id"].count()

#Revenue per month
df.groupby("month")["total_amount"].sum()

#Revenue per quarter
df["quarter"] = df["order_date"].dt.quarter
df.groupby("quarter")["total_amount"].sum()


,total_amount
quarter,
1,2010626.64
2,1945348.72
3,2037385.10
4,2018263.66


In [3]:
#USER BEHAVIOR PATTERNS

#Total spend per user
df.groupby("user_id")["total_amount"].sum()

#Average order value per user
df.groupby("user_id")["total_amount"].mean()

#Number of orders per user
df.groupby("user_id")["order_id"].count().sort_values(ascending=False)

#Repeat customers
df.groupby("user_id")["order_id"].count().gt(1).sum()

np.int64(2539)

In [4]:
#CITY-WISE PERFORMANCE

#Revenue by city
df.groupby("city")["total_amount"].sum()

#Orders by city
df.groupby("city")["order_id"].count()

#Average order value by city
df.groupby("city")["total_amount"].mean()

,total_amount
city,
Bangalore,802.234308
Chennai,806.202118
Hyderabad,803.985779
Pune,792.097914


In [5]:
#CUISINE-WISE PERFORMANCE

#Revenue by cuisine
df.groupby("cuisine")["total_amount"].sum()

#Orders by cuisine
df.groupby("cuisine")["order_id"].count()

#Average order value by cuisine
df.groupby("cuisine")["total_amount"].mean()

,total_amount
cuisine,
Chinese,798.389020
Indian,798.466011
Italian,799.448578
Mexican,808.021344


In [6]:
#MEMBERSHIP IMPACT (Gold vs Regular)

#Orders split
df.groupby("membership")["order_id"].count()

#Revenue split
df.groupby("membership")["total_amount"].sum()

#Average order value comparison
df.groupby("membership")["total_amount"].mean()

,total_amount
membership,
Gold,797.145556
Regular,805.158434


In [7]:
#Which city has the highest total revenue (total_amount) from Gold members?
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"].sum() \
    .idxmax()


'Chennai'

In [8]:
#Which cuisine has the highest average order value across all orders?
df.groupby("cuisine")["total_amount"].mean().idxmax()


'Mexican'

In [9]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?
user_spend = df.groupby("user_id")["total_amount"].sum()
user_spend[user_spend > 1000].nunique()


2543

In [10]:
#Which restaurant rating range generated the highest total revenue?
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[0,3.5,4.0,4.5,5.0],
    labels=["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"]
)
df.groupby("rating_range")["total_amount"].sum().idxmax()


/tmp/ipython-input-2760204496.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().idxmax()


'4.6–5.0'

In [11]:
df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,month,quarter,rating_range
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,2023-02,1,3.0–3.5
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,2023-01,1,4.1–4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,2023-07,3,3.6–4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,2023-10,4,4.6–5.0
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,2023-12,4,3.0–3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,2023-05-21,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7,2023-05,2,4.6–5.0
9996,9997,2867,267,2023-08-06,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2,2023-08,3,4.1–4.5
9997,9998,522,420,2023-11-11,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0,2023-11,4,3.6–4.0
9998,9999,319,492,2023-09-08,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0,2023-09,3,3.6–4.0


In [12]:
#Among Gold members, which city has the highest average order value?
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"].mean() \
    .idxmax()


'Chennai'

In [13]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?
df.groupby("cuisine").agg(
    restaurant_count=("restaurant_id", "nunique"),
    revenue=("total_amount", "sum")
).sort_values(
    by=["restaurant_count", "revenue"],
    ascending=[True, False]
).head(1)


,restaurant_count,revenue
cuisine,,
Chinese,120,1930504.65


In [14]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)
round(df[df["membership"] == "Gold"].shape[0] / df.shape[0] * 100)

50

In [15]:
#Which restaurant has the highest average order value but less than 20 total orders?
df.groupby("restaurant_name_y").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
).query(
    "total_orders < 20"
).sort_values(
    by="avg_order_value", ascending=False
).head(1)



,avg_order_value,total_orders
restaurant_name_y,,
Restaurant_294,1040.222308,13


In [16]:
#Which combination contributes the highest revenue?

df.groupby(
    ["membership","cuisine"]
)["total_amount"].sum().idxmax()


('Regular', 'Mexican')

In [17]:
#During which quarter of the year is the total revenue highest?
df["quarter"] = df["order_date"].dt.quarter
df.groupby("quarter")["total_amount"].sum().idxmax()

np.int32(3)

In [18]:
# How many total orders were placed by users with Gold membership?
df[df["membership"] == "Gold"].shape[0]

4987

In [19]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())



1889367

In [20]:
# How many distinct users placed at least one order?
df["user_id"].nunique()


2883

In [21]:
#What is the average order value (rounded to 2 decimals) for Gold members?
round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [22]:
#How many orders were placed for restaurants with rating ≥ 4.5?
df[df["rating"] >= 4.5]["order_id"].count()


np.int64(3374)

In [23]:
#How many orders were placed in the top revenue city among Gold members only?
top_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

print(df[(df["membership"] == "Gold") & (df["city"] == top_city)]["order_id"].count())

1337


In [24]:
#The column used to join orders.csv and users.json is __________.
inner_merge = pd.merge(orders, users, on="user_id", how="inner")  #ANS:- user_id

#The dataset containing cuisine and rating information is stored in __________ format.
#ANS:- restaurants.sql

#The total number of rows in the final merged dataset is __________.
df.tail() #ANS:- 10000

#If a user has no matching record in users.json, the merged values will be __________.
#ANS:- NULL/NAN

#The Pandas function used to combine datasets based on a key is __________.
pd.merge(orders, users, on="user_id", how="inner")  #ANS:- merge()

#The column membership in the final dataset originates from the __________ file
#ANS:-users.json

#The join key used to combine orders data with restaurant details is __________.
#ANS:-orders.restaurant_id to restaurants.restaurant_id

#The column that helps identify the type of food served by a restaurant is __________.
#ANS:-cuisine

#If a user places multiple orders, their personal details appear __________ times in the final merged dataset.
#ANS:- once per order

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold
